## A second-level heading
Computational Theory Problems

In [12]:
import numpy as np

## Problem 1: Binary Words and Operations

In [21]:
def parity(x, y, z):
    # bitwise XOR
    ## Example 1 0 1, if there are odd number of 1s the result is 1
    ## if there are even number of 1s the result is 0
    ## result of 1 0 1 would be 0 as there are an even number of 1s
    return x ^ y ^ z

print(parity(0b1100, 0b1111, 0b1000))


11


The `numpy.array` function ([see official documentation](https://numpy.org/doc/stable/reference/generated/numpy.array.html)) creates an array from a list or other data structure.

In [14]:
a = np.array([0, 1, 0, 1])
b = np.array([1, 1, 0, 1])
c = np.array([0, 0, 0, 1])

print(parity(a, b , c))

[1 0 0 1]


In [19]:
def Ch(x, y ,z):
    return (x & y) ^ (~x & z)

print(Ch(1001, 1100, 1010))

90


## Problem 2: Fractional Parts of Cube Roots

## Problem 3: Padding


## Problem 4: Hashes

## Problem 5: Passwords